## Visualización de tasas de mortalidad infantil por desnutrición (Colombia, nivel municipal)

#### Importación de librerias

In [ ]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import mapclassify
import geopandas as gpd


#### Importación de los datos

In [ ]:
df=pd.read_csv("Data/tasa_mortalidad_menores_5.csv",dtype={"MPIO":str})

### Mapas de Colombia

#### Municipios

El shapefile de municipios no está incluido en este repositorio por su tamaño. Puedes descargarlo desde el siguiente enlace:

[Descargar shapefile de municipios](https://geoportal.dane.gov.co/descargas/mgn_2024/MGN2024_MPIO_POLITICO.zip)

Una vez descargado, colóca todos los archivos en la carpeta `Municipios/` del proyecto.


In [ ]:
municipios=gpd.read_file("Municipios/MGN_ADM_MPIO_GRAFICO.shp")

#### Departamentos

El shapefile de **departamentos** fue cargado con los siguientes propósitos:

1. **Contextualizar geográficamente**  
   Aunque el análisis se realiza a nivel municipal, la visualización se beneficia al mostrar también los límites departamentales. Esto permite al lector ubicar fácilmente en qué parte del país se encuentran los municipios con mayores tasas de desnutrición infantil.

2. **Referencia visual y jerárquica**  
   Dado que los departamentos son unidades administrativas superiores, incluir su contorno facilita una lectura más estructurada del mapa. Permite identificar patrones regionales o concentraciones dentro de determinadas zonas del país (como regiones Caribe, Pacífica, etc.).

3. **Diseño estético del mapa**  
   Superponer los límites departamentales sobre el mapa de municipios mejora la calidad visual del gráfico, delimitando con claridad los espacios sin recargar la información principal.

4. **Posibles futuros análisis agregados**  
   En caso de necesitar agrupar tasas o muertes a nivel departamental (por ejemplo, para análisis comparativos entre departamentos), ya se cuenta con la geometría necesaria para hacerlo fácilmente.


In [ ]:
departamentos=gpd.read_file("Departamentos/MGN_ADM_DPTO_POLITICO.shp")
departamentos=departamentos.to_crs(municipios.crs)

### Preparación del shapefile para el cruce con los datos de mortalidad infantil ( 0 a 4) por desnutricion

El shapefile contiene la geometría de los municipios de Colombia, los códigos municipales se encuentran en la columna (`mpio_cdpmp`). Para realizar el cruce con el DataFrame que contiene las tasas de suicidio por departamento, fue necesario:

**Renombrar la columna a `MPIO`** para que coincida con el nombre usado en los datos de tasas.

In [ ]:
municipios.rename(columns={'mpio_cdpmp':'MPIO'}, inplace=True)

### Unión del shapefile con los datos de tasas de mortalidad infantil por desnutrición

Una vez que tanto el shapefile como el DataFrame de tasas cuentan con una columna en común (`MPIO`) y tienen el mismo tipo de dato, se realiza la unión (`merge`) entre ambos. La columna `MPIO` corresponde al código DIVIPOLA, que es el identificador oficial asignado por el DANE a cada municipio en Colombia.

Esta unión permite incorporar al shapefile las tasas de mortalidad infantil por desnutrición, por municipio y año, y así poder visualizarlas geográficamente.



In [ ]:
figura= pd.merge(municipios, df, how="left", on='MPIO')

## Selección de municipios para el análisis

Para enfocar el análisis en contextos donde las cifras son estadísticamente significativas y epidemiológicamente relevantes, se aplicaron dos criterios de filtrado:

1. **Tamaño poblacional**  
   Se seleccionaron únicamente los municipios con una población de **5.000 o más niños menores de 5 años**. Este umbral permite excluir municipios con poblaciones muy pequeñas, donde incluso una sola muerte puede generar tasas extremadamente elevadas pero poco representativas.

2. **Tasa de mortalidad por desnutrición**  
   Se incluyeron solamente aquellos municipios cuya tasa de mortalidad por desnutrición en menores de 5 años es **superior a 1 por cada 100.000 niños**.

   De esta manera, el análisis se centra en municipios donde el problema representa una preocupación real de salud pública y no una fluctuación aleatoria en municipios con muy baja población.

In [ ]:
figura= figura[(figura['Población_menor']>=5000) & (figura['tasa']>=1)]

### Mapa Corropletico

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# === 1. Años disponibles ===
years = sorted(figura["AÑO"].unique())

# === 2. Clasificador de tasas personalizado ===
bins = [3, 5, 10, 20, 30, 50, 100, figura["tasa"].max() + 1]

# === 3. Crear figura base ===
fig, ax = plt.subplots(figsize=(12, 10))

# === 4. Función de animación por año ===
def update(year):
    ax.clear()

    # Subconjunto del año
    data = figura[figura["AÑO"] == year]

    # Mapa de municipios
    data.plot(
        column='tasa',
        cmap='OrRd',
        linewidth=0.5,
        edgecolor='0.8',
        legend=True,
        scheme='UserDefined',
        classification_kwds={'bins': bins},
        legend_kwds={
            'title': 'Tasas\n(menores de 5 años)',
            'loc': 'upper left',
            'bbox_to_anchor': (1, 0.4),
            'fontsize': 10
        },
        ax=ax
    )

    # Contorno de departamentos
    departamentos.boundary.plot(ax=ax, linewidth=1.2, edgecolor='gray')

    # === Etiquetas con flechas (solo tasa >= 45) ===
    offset_dict = {
        "AGUSTÍN CODAZZI": (1.8, 0.3),
        "CIÉNAGA": (-0.9, 0.7),
        "CUMARIBO": (-1.5, -0.7),
        "EL BANCO": (0.5, -0.6),
        "FUNDACIÓN": (0.4, 0.6),
        "LETICIA": (-1, -1),
        "MAICAO": (1.9, 0.2),
        "MANAURE": (-0.3, 1.5),
        "PUERTO GAITÁN": (2, 0.6),
        "QUIBDÓ": (-1.8, -0.3),
        "RIOHACHA": (-1.1, 1.1),
        "TIBÚ": (1.5, -0.2),
        "TIERRALTA": (-2, 1),
        "TURBO": (-1.8, -0.5),
        "URIBIA": (1.3, 1.2),
        "ZONA BANANERA": (-2, 0)
    }

    for _, row in data.iterrows():
        if row["tasa"] < 45 or row.geometry.is_empty:
            continue

        nombre = row["mpio_cnmbr"]
        if nombre not in offset_dict:
            continue

        x, y = row.geometry.representative_point().coords[0]
        dx, dy = offset_dict[nombre]

        etiqueta = f"{nombre} ({row['tasa']:.1f})"

        ax.annotate(
            etiqueta,
            xy=(x, y),
            xytext=(x + dx, y + dy),
            fontsize=9,
            ha='center',
            va='center',
            color='black',
            arrowprops=dict(
                arrowstyle="->",
                color='yellow',
                lw=1.5
            ),
            bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="black", lw=0.5)
        )

    # Título fijo del mapa
    ax.set_title("Tasas de mortalidad infantil ( 0-4 años) por desnutrición en los municipios de Colombia", fontsize=16, loc='center', pad=20)

    # Año grande a la derecha del mapa
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.text(
        xlim[1] + 1,  # posición fuera del mapa
        (ylim[0] + ylim[1]) / 2,  # centrado vertical
        str(year),
        fontsize=40,
        weight='bold',
        ha='left',
        va='center'
    )

    ax.axis('off')

    # Pie de página
    fig.text(
        0.01, 0.01,
        "Elaboración propia.\n"
        "Tasa de muertes por desnutrición en menores de 5 años por 100.000 hab.\n"
        "Fuente: Microdatos Estadísticas Vitales – DANE.\n"
        "Nota: Se excluyen municipios con población menor a 5.000 niños (0-4 años), y tasas menores a 1.\n"
        "Se etiquetan únicamente los municipios con tasas mayores o iguales a 45 por 100.000 hab (0 a 4 años).",
        ha='left', va='bottom', fontsize=9, color='gray'
    )

    plt.tight_layout()
    plt.close()

# === 5. Crear animación ===
ani = animation.FuncAnimation(fig, update, frames=years, interval=1500, repeat=False)

# === 6. Mostrar en notebook ===
HTML(ani.to_jshtml())


In [ ]:
ani.save("mortalidad_desnutricion.mp4", writer='ffmpeg', dpi=200)